In [2]:
import pandas as pd
#import glob

#path = r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE' 
#all_files = glob.glob(path + "/*.csv")

#li = []

#for filename in all_files:
   # df = pd.read_csv(filename, index_col=None, header=0)
   # li.append(df)

#frame = pd.concat(li, axis=0, ignore_index=True)
df0 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')
df1 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv')

df2 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv')

df3 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv')

df4 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv')

df5 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv')
df6 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv')

df7 = pd.read_csv(r'C:\Users\abhiw\Downloads\cicids2017\MachineLearningCSV\MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv')

print(df0.shape)
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)
print(df7.shape)

data = [df0,df1,df2,df3,df4,df5,df6,df7]
df  = pd.concat(data)

print("Read {} rows.".format(len(df)))
df.dropna(inplace=True,axis=1)

pd.set_option('display.max_columns', 5)
pd.set_option('display.max_rows', 5)
# display 5 rows
display(df[0:5])

(225745, 79)
(286467, 79)
(191033, 79)
(529918, 79)
(288602, 79)
(170366, 79)
(445909, 79)
(692703, 79)
Read 2830743 rows.


,Destination Port,Flow Duration,...,Idle Min,Label
0,54865,3,...,0,BENIGN
1,55054,109,...,0,BENIGN
2,55055,52,...,0,BENIGN
3,46236,34,...,0,BENIGN
4,54863,3,...,0,BENIGN


In [3]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

def expand_categories(values):
    result = []
    s = values.value_counts()
    t = float(len(values))
    for v in s.index:
        result.append("{}:{}%".format(v,round(100*(s[v]/t),2)))
    return "[{}]".format(",".join(result))
        
def analyze(df):
    print()
    cols = df.columns.values
    total = float(len(df))

    print("{} rows".format(int(total)))
    for col in cols:
        uniques = df[col].unique()
        unique_count = len(uniques)
        if unique_count>100:
            print("** {}:{} ({}%)".format(col,unique_count,int(((unique_count)/total)*100)))
        else:
            print("** {}:{}".format(col,expand_categories(df[col])))
            expand_categories(df[col])

In [4]:
analyze(df)


2830743 rows
**  Destination Port:53805 (1%)
**  Flow Duration:1050899 (37%)
**  Total Fwd Packets:1432 (0%)
**  Total Backward Packets:1747 (0%)
** Total Length of Fwd Packets:17928 (0%)
**  Total Length of Bwd Packets:64698 (2%)
**  Fwd Packet Length Max:5279 (0%)
**  Fwd Packet Length Min:384 (0%)
**  Fwd Packet Length Mean:99716 (3%)
**  Fwd Packet Length Std:253909 (8%)
** Bwd Packet Length Max:4838 (0%)
**  Bwd Packet Length Min:583 (0%)
**  Bwd Packet Length Mean:147614 (5%)
**  Bwd Packet Length Std:248869 (8%)
**  Flow Packets/s:1240164 (43%)
**  Flow IAT Mean:1166311 (41%)
**  Flow IAT Std:1056642 (37%)
**  Flow IAT Max:580289 (20%)
**  Flow IAT Min:136316 (4%)
** Fwd IAT Total:493098 (17%)
**  Fwd IAT Mean:737737 (26%)
**  Fwd IAT Std:700313 (24%)
**  Fwd IAT Max:437316 (15%)
**  Fwd IAT Min:110631 (3%)
** Bwd IAT Total:414928 (14%)
**  Bwd IAT Mean:670824 (23%)
**  Bwd IAT Std:709042 (25%)
**  Bwd IAT Max:368285 (13%)
**  Bwd IAT Min:66074 (2%)
** Fwd PSH Flags:[0:95.36%,1

In [5]:
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd
    
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


In [6]:
for i in df.columns:
    if i ==' Label':
        continue
    else:
        encode_numeric_zscore(df, i)
        
df.dropna(inplace=True,axis=1)
df[0:5]

x_columns = df.columns.drop(' Label')
x = df[x_columns].values
dummies = pd.get_dummies(df[' Label'])
outcomes = dummies.columns
num_classes = len(outcomes)
y = dummies.values

In [7]:
df.groupby(' Label')[' Label'].count()

 Label
BENIGN                        2273097
Bot                              1966
                               ...   
Web Attack � Sql Injection         21
Web Attack � XSS                  652
Name:  Label, Length: 15, dtype: int64

In [8]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
                        patience=5, verbose=1, mode='auto',
                           restore_best_weights=True)
model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=1000)
        

Train on 2123057 samples, validate on 707686 samples
Epoch 1/1000
2123057/2123057 - 283s - loss: 0.2189 - val_loss: 0.1620
Epoch 2/1000
2123057/2123057 - 217s - loss: 0.1634 - val_loss: 0.1495
Epoch 3/1000
2123057/2123057 - 207s - loss: 0.1534 - val_loss: 0.1462
Epoch 4/1000
2123057/2123057 - 196s - loss: 0.1460 - val_loss: 0.1405
Epoch 5/1000
2123057/2123057 - 210s - loss: 0.1384 - val_loss: 0.1689
Epoch 6/1000
2123057/2123057 - 204s - loss: 0.1293 - val_loss: 0.1212
Epoch 7/1000
2123057/2123057 - 229s - loss: 0.1310 - val_loss: 0.1150
Epoch 8/1000
2123057/2123057 - 229s - loss: 0.1284 - val_loss: 0.1267
Epoch 9/1000
2123057/2123057 - 219s - loss: 0.1252 - val_loss: 0.1178
Epoch 10/1000
2123057/2123057 - 231s - loss: 0.1225 - val_loss: 0.1224
Epoch 11/1000
2123057/2123057 - 231s - loss: 0.1219 - val_loss: 0.1217
Epoch 12/1000
2123057/2123057 - 226s - loss: 0.1207 - val_loss: 0.1108
Epoch 13/1000
2123057/2123057 - 224s - loss: 0.1187 - val_loss: 0.0996
Epoch 14/1000
2123057/2123057 - 2

In [9]:
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9768343587410235
